In [ ]:
import altair as alt
import pandas as pd

combined_val_df = pd.read_csv('predictive_accuracy.csv')

x_axis_labels = {
    'Age_in_Yrs': 'Age',
    'CogCrystalComp_Unadj': 'Crystalized Intelligence',
    'CogFluidComp_Unadj': 'Fluid Intelligence',
    'CogTotalComp_Unadj': 'Global Intelligence',
    'DDisc_AUC_200': 'Impulsivity',
    'Endurance_Unadj': 'Endurance',
    'IWRD_TOT': 'Verbal Memory',
    'ReadEng_Unadj': 'Reading Ability',
    'SCPT_SEN': 'Attention',
    'VSPLOT_TC': 'Spatial Orientation'}

combined_val_df['variable'] = combined_val_df['variable'].replace(x_axis_labels)

algos = ["Local Conn.", "TP-SGL", "TP-Lasso"]

combined_val_df.data = combined_val_df.data.replace({
    "lc": algos[0],
    "tract-sgl": algos[1],
    "tracts": algos[2]})

box_plot = alt.Chart(combined_val_df).mark_boxplot(opacity=0.5, outliers=False).encode(
    y=alt.Y('data:N', axis=None),
    x=alt.X('value:Q', title='Predictive Accuracy (R-Squared)'),
    color=alt.Color('data:N', legend=alt.Legend(title=""),
                    scale=alt.Scale(domain=algos, range=["red", "green", "blue"])),
)

points = alt.Chart(combined_val_df).mark_point(size=10, filled=True).encode(
    y=alt.Y('data:N'),
    x=alt.X('value:Q'),
    yOffset="jitter:Q",
    color=alt.Color('data:N'),
).transform_calculate(
    jitter="sqrt(-2*log(random()))*cos(2*PI*random())"
)

combined_chart = (box_plot + points)

font_size = 15

combined_chart = combined_chart.facet(
    row=alt.Row(
        "variable",
        title='Behavioral Phenotype',
        header=alt.Header(
            labelFontSize=font_size,
            titleFontSize=font_size,
            labelAngle=0,
            labelPadding=10,
            labelAlign='left'))
)

final_chart = combined_chart.configure_view(
    strokeWidth=0
).configure_axis(
    labelFontSize=font_size,
    titleFontSize=font_size,
    labelLimit=0
).configure_title(
    fontSize=font_size
).configure_legend(
    labelFontSize=font_size,
    titleFontSize=font_size
)

final_chart.save('lc_vs_tract-lasso-sgl.png', ppi=300)
final_chart
